In [1]:
import pandas as pd
import folium
import json
import numpy as np
import requests

In [2]:
# df_trans=pd.read_csv('./연평균 지하철 이용객.csv')

In [3]:
# # kakao key 불러오기
# with open('C:/Users/tjoeun/Desktop/송수린/workspace/kakaokey.txt') as file:
#     kakao_key = file.read()

# # kakao_key를 통해서 모든 지하철역 주소 위도 경도 좌표 구하기
# url = "https://dapi.kakao.com/v2/local/search/keyword.json?"
# header = {"Authorization": "KakaoAK " + kakao_key}
# addr, lngs, lats = [], [], []
# for i in range(len(df_trans)):
#     query = df_trans['지하철역'][i]
#     if '역' not in df_trans['지하철역'][i]:
#         station = df_trans['지하철역'][i]
#         if '(' in df_trans['지하철역'][i]:
#             station = df_trans['지하철역'][i].split('(')[0]
#         query = station + '역'
#     result = requests.get(url, params = {'query':query}, headers={'Authorization' : 'KakaoAK ' + kakao_key}).json()['documents']
#     try:
#         addr.append(result[0]['address_name'])
#         lngs.append(float(result[0]['x']))
#         lats.append(float(result[0]['y']))
#     except:
#         print(df_trans['지하철역'][i])

In [4]:
# df_trans['주소'] = pd.DataFrame(addr)
# df_trans['위도'] = pd.DataFrame(lats)
# df_trans['경도'] = pd.DataFrame(lngs)

In [5]:
# df_trans_seoul = df_trans[df_trans['주소'].str.contains('서울')]

In [6]:
# df_trans_seoul.set_index('지하철역', inplace=True)

In [7]:
# df_trans_seoul.reset_index(inplace=True)

In [8]:
# df_trans_seoul.to_csv('seoul_stations.csv', index=False)

In [9]:
df_trans_seoul=pd.read_csv('./seoul_stations.csv')

In [10]:
df_trans_seoul

,지하철역,호선명,04시-05시 승차인원,04시-05시 하차인원,05시-06시 승차인원,05시-06시 하차인원,06시-07시 승차인원,06시-07시 하차인원,07시-08시 승차인원,07시-08시 하차인원,...,1시이후 승차,1시이후 하차,총 이용객,유임승차인원,무임승차인원,유임하차인원,무임하차인원,주소,위도,경도
0,가산디지털단지,1호선,1340.000000,6.133333,8205.400000,1980.583333,7914.766667,10941.750000,14763.350000,46802.533333,...,0.700000,2.033333,9.784055e+05,418268.766667,29773.750000,497380.250000,32982.716667,서울 금천구 가산동 468-4,37.480396,126.882662
1,개봉,1호선,1226.050000,8.633333,18414.966667,5494.800000,36569.333333,11855.966667,95251.200000,16530.883333,...,0.300000,2.283333,1.322553e+06,514742.366667,145080.383333,511793.650000,150936.250000,서울 구로구 개봉동 415,37.494642,126.858716
2,광운대,1호선,1891.300000,9.366667,6056.233333,1672.750000,13794.750000,3227.133333,35362.750000,6220.833333,...,0.333333,32.966667,5.096399e+05,209971.700000,50769.800000,198687.033333,50211.316667,서울 노원구 월계동 85,37.623837,127.061695
3,구로,1호선,10105.066667,37.166667,24259.666667,4950.850000,20134.566667,12746.650000,39252.016667,32344.650000,...,1.016667,73.483333,1.146417e+06,476611.116667,88107.983333,496615.200000,85082.766667,서울 구로구 구로동 585-3,37.503342,126.882308
4,구일,1호선,255.433333,8.933333,3688.483333,1420.150000,9397.683333,3487.966667,27301.400000,7873.316667,...,0.100000,0.616667,4.307922e+05,176276.350000,36079.116667,182617.083333,35819.650000,서울 구로구 구로동 636-45,37.496255,126.869691
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
373,이촌(국립중앙박물관),중앙선,73.483333,5.566667,1132.516667,613.483333,3676.050000,2337.016667,8548.933333,4727.466667,...,0.150000,0.300000,2.137298e+05,94778.033333,26604.483333,70337.533333,22009.800000,서울 용산구 이촌동 17-69,37.522413,126.973544
374,중랑,중앙선,106.666667,2.466667,5928.416667,500.233333,9468.983333,2349.000000,22974.583333,3476.983333,...,0.016667,0.416667,3.225320e+05,117228.316667,47920.950000,110364.683333,47018.083333,서울 중랑구 중화동 73-7,37.594843,127.075680
375,청량리(서울시립대입구),중앙선,1166.816667,10.450000,6484.983333,1232.766667,8103.333333,10701.366667,25543.000000,13846.966667,...,0.233333,8.333333,9.497495e+05,321003.316667,136824.166667,347664.916667,144257.100000,서울 동대문구 전농동 591-53,37.580733,127.048504
376,한남,중앙선,124.983333,0.983333,1977.766667,635.500000,2819.466667,4710.166667,5202.366667,9678.383333,...,0.050000,0.283333,2.408121e+05,92782.833333,29769.633333,90481.183333,27778.416667,서울 용산구 한남동 399-7,37.529429,127.009209


In [11]:
df_map=pd.concat([df_trans_seoul.iloc[:,:2],df_trans_seoul.iloc[:,-3:]], axis=1)

In [12]:
df_map

,지하철역,호선명,주소,위도,경도
0,가산디지털단지,1호선,서울 금천구 가산동 468-4,37.480396,126.882662
1,개봉,1호선,서울 구로구 개봉동 415,37.494642,126.858716
2,광운대,1호선,서울 노원구 월계동 85,37.623837,127.061695
3,구로,1호선,서울 구로구 구로동 585-3,37.503342,126.882308
4,구일,1호선,서울 구로구 구로동 636-45,37.496255,126.869691
...,...,...,...,...,...
373,이촌(국립중앙박물관),중앙선,서울 용산구 이촌동 17-69,37.522413,126.973544
374,중랑,중앙선,서울 중랑구 중화동 73-7,37.594843,127.075680
375,청량리(서울시립대입구),중앙선,서울 동대문구 전농동 591-53,37.580733,127.048504
376,한남,중앙선,서울 용산구 한남동 399-7,37.529429,127.009209


In [13]:
# 환승 가능한 역들
df_tran=[]
for i in df_trans_seoul['지하철역'].unique():
    index = df_trans_seoul[df_trans_seoul['지하철역'] == i].index
    if len(df_trans_seoul[df_trans_seoul['지하철역'] == i]) > 1:
        for j in index:
            df_tran.append(df_trans_seoul.iloc[j])
    else:
        pass

In [14]:
df_tran=pd.concat([pd.DataFrame(df_tran).iloc[:,:2],pd.DataFrame(df_tran).iloc[:,-3:]], axis=1)

In [15]:
df_tran=df_tran.reset_index()

In [16]:
#  01~09호선 분리
df_map_1=df_map[df_map['호선명']=='1호선']
df_map_2=df_map[df_map['호선명']=='2호선']
df_map_3=df_map[df_map['호선명']=='3호선']
df_map_4=df_map[df_map['호선명']=='4호선']
df_map_5=df_map[df_map['호선명']=='5호선']
df_map_6=df_map[df_map['호선명']=='6호선']
df_map_7=df_map[df_map['호선명']=='7호선']
df_map_8=df_map[df_map['호선명']=='8호선']
df_map_9=df_map[df_map['호선명']=='9호선']
df_map_10=df_map[df_map['호선명']=='경의선']
df_map_11=df_map[df_map['호선명']=='경춘선']
df_map_12=df_map[df_map['호선명']=='공항철도 1호선']
df_map_13=df_map[df_map['호선명']=='분당선']
df_map_14=df_map[df_map['호선명']=='우이신설선']
df_map_15=df_map[df_map['호선명']=='중앙선']

In [17]:
map=folium.Map(location=[df_map.위도.mean(),df_map.경도.mean()], zoom_start=11,
               tiles='Stamen Terrain') 

r = requests.get('https://raw.githubusercontent.com/southkorea/seoul-maps/master/kostat/2013/json/seoul_municipalities_geo_simple.json')
c = r.content
seoul_geo = json.loads(c)

map.choropleth(geo_data=seoul_geo,
             line_color='black',
             fill_opacity=0.1,
             line_opacity=1,
             line_weight=2
            )

colors=['darkblue','darkgreen','darkorange','darkturquoise','slateblue','chocolate','olive',
      'dippink','tan','mediumaquamarine','forestgreen','dodgeblue','gold','yellogreen','mediumaquamarine']


for i in range(1,15):
    for j in globals()[f'df_map_{i}'].index:
        folium.Circle(
        radius=200, 
        location=[globals()[f'df_map_{i}'].위도[j],globals()[f'df_map_{i}'].경도[j]],
        popup=folium.Popup(globals()[f'df_map_{i}'].호선명[j], max_width=200),
        tooltip=globals()[f'df_map_{i}'].지하철역[j],
        color=colors[i-1],
        fill=True,
        fill_color=colors[i-1]
    ).add_to(map)

for i in df_tran.index: 
    folium.Circle(
        radius=200, 
        location=[df_tran.위도[i],df_tran.경도[i]],
        popup=folium.Popup(df_tran.호선명[i], max_width=200),
        tooltip=df_tran.지하철역[i]+'(환승가능역)',
        color='red',
        fill=True,
        fill_opacity=1,
        fill_color='red'
    ).add_to(map)
    
title='<h3 align="center" style="font-size:20px">서울시 지하철 노선</h3>'
map.get_root().html.add_child(folium.Element(title))

map

c:\python38\lib\site-packages\folium\folium.py:465: FutureWarning: The choropleth  method has been deprecated. Instead use the new Choropleth class, which has the same arguments. See the example notebook 'GeoJSON_and_choropleth' for how to do this.
  warnings.warn(


NameError: name 'candidate_location' is not defined

In [ ]:
candidate=pd.read_csv('./환승역 후보.csv')

In [ ]:
candidate

In [ ]:
candidate_location=[]
for i in range(0,len(candidate['지하철역'])):
    candidate_location=pd.concat([pd.DataFrame(candidate_location),df_map[df_map['지하철역']==candidate['지하철역'][i]]], axis=0)

In [ ]:
candidate_location=candidate_location.reset_index()
candidate_location.drop(columns='index', inplace=True)

In [ ]:
candidate_location

In [ ]:
# 환승 후보역 지도에 표시
for i in candidate_location.index: 
    folium.Marker(
        radius=200, 
        location=[candidate_location.위도[i],candidate_location.경도[i]],
        popup=folium.Popup(candidate_location.호선명[i], max_width=200),
        tooltip=candidate_location.지하철역[i]+'(환승후보역)',
        color='yellow',
        icon = folium.Icon(
        icon='train-subway')
    ).add_to(map)